# 8장 이미지를 위한 인공신경망

8장에서는 이미지데이터를 위한 인공신경망 중에서 **CNN(convolutional neural network)**을 공부한다. 보통 'convolution'을 '합성곱'이라고 번역하므로 CNN은 **'합성곱신경망'**이라고 부른다. 수학, 통계학에서 쓰는 합성곱의 설명은 [이곳](https://ko.wikipedia.org/wiki/%ED%95%A9%EC%84%B1%EA%B3%B1)에서 볼 수 있다.

CNN에 이어서 교재 마지막에 있는 9장에서는 텍스트데이터 분석을 위한 신경망 중에서 **RNN(recurrent neural network)**, 즉 '순환신경망'을 공부한다.

## 8-1 CNN의 구성 요소

### 8-1-1 합성곱

합성곱 (교재 423쪽): 입력 데이터에 마법의 도장을 찍어서 유용한 특성만 드러나게 하는 방법 (???)

1) 1차원 입력 데이터의 합성곱

10개의 입력 데이터로 밀집층을 만든다고 해보자.

밀집층의 뉴런들 중에서 하나만 나타낸다면 423쪽 그림과 같을 것이다. 7장에서 공부한 방법에 따르면 손실함수를 작게 만드는 10개의 가중치 $w_1 , w_2 , \ldots, w_{10} $와 절편 $b$를 찾기 위해 epoch를 반복하면서 경사하강법을 이용할 것이다.

그런 작업은 밀집층에 있는 뉴런의 수(우리가 정해야하는 하이퍼 파라미터 중 하나다)만큼 반복된다. 즉 밀집층의 출력은 뉴런의 수만큼 만들어지므로 입력층 데이터의 수와는 아무 상관이 없다.

423쪽 그림을 424쪽 그림과 비교해보자.

역시 10개의 입력 데이터가 있는데 이번에는
전체 데이터가 아니라 순서대로 위에서 3개씩만 가지고 출력을 하나씩 만든다고 해보자. 동일한 $w_1 , w_2 , w_3 $과 $b$를 이용해서 가능한 출력을 모두 만든다면 출력은 몇 개 생길까? 그 수는 역시 우리가 마음대로 정할 수 있는 하이퍼 파라미터일까?

출력 수는 8개인데 이 숫자는 우리가 바꾸거나 정할 수 없다. 즉 이번에는 출력의 수가 하이퍼파라미터가 아니다.

우리가 정할 수 있는 하이퍼파라미터는 출력의 수가 아니라 출력을 하나 계산할 때 이용하는 데이터의 수다. 따라서 그런 출력을 7장의 밀집층에서와 같이 '뉴런'이라고 부르기는 이상하므로 합성곱 신경망에서는 출력을 **'필터' 또는 '커널'**이라고 부른다.

2) 2차원 입력 데이터의 합성곱

이번에는 입력데이터가 1차원이 아닌 4행 4열로 된 2차원 데이터라고 하자 (426쪽 그림).

데이터 수는 16개인데 그 가운데 3행 3열(9개) 필터로 출력을 만들어 보자. 즉 필터는 가중치 아홉 개($w_1, w_2 , \ldots, w_9 $)와 절편 $b$ 하나로 출력을 만든다.

이런 2차원 합성곱 커널 출력이 몇 개 생길까? 426쪽 그림과 같이 모두 4개 생긴다.

이렇게 만든 4개의 출력을 427쪽 그림과 같이 $2 \times 2$ 배열처럼 만든 것을 **'특성 맵(feature map)'**이라고 부른다.

여기서는 $4 \times 4$ 입력에 $ 3 \times 3$ 커널을 적용하여 $ 2 \times 2 $ 특성맵을 만들었다.

위와 같은 작업을 다른 가중치를 가진 필터로 여러 번 반복할 수 있다. 428쪽 그림이 다른 필터(다른 색깔로 표현) 세 개로 만든 특성 맵 세 개를 보여준다.

여기서 우리는 8장의 제목이 <이미지를 위한 인공신경망>으로 붙은 이유를 짐작할 수 있다. 7장에서는 $28 \times 28$ 2차원 이미지 데이터를 reshape()
또는 Flatten을 이용해서 1차원으로 바꾸어서 분석했지만 CNN에서는 이미지의 2차원 형태를 그대로 유지하므로 더 성능이 좋아진다.

### 8-1-2 케라스 합성곱 층 Conv2D

1) Conv2D

7장에서 공부한 밀집층을 만드는 클래스와 마찬가지로 합성곱층을 만드는 케라스 클래스도 `keras.layers` 안에 들어있다. 클래스의 이름은 `Conv2D`이고 사용 방법은 `Dense()`와 비슷하지만 커널의 크기, 즉 `kernel_size`를 지정하는 점이 다르다. 428쪽의 코드를 잠깐 보자.

    from tensorflow import keras
    keras.layers.Conv2D(10, kernel_size = (3, 3), activation = 'relu')

여기서 숫자 10은 필터의 수이고 (3, 3)은 커널의 크기다.

2) 패딩과 스트라이드

a) padding

앞에서는 $4 \times 4$ 입력에 $ 3 \times 3$ 커널을 적용하여 $ 2 \times 2 $ 특성맵을 만들었다. $4 \times 4$ 입력에 $ 3 \times 3$ 커널을 적용하여 입력과 같은 크기의 $4 \times 4$ 특성맵을 만들 수 있을까?

430쪽 위의 그림처럼 $4 \times 4$ 입력의 주위를 가상의 값(보통 0)으로 채워서 마치 $6 \times 6$인 것처럼 만들고 $ (3, 3)$ 크기의 커널을 적용하면 4행, 4열 모두 16개로 이루어진 특성맵을 얻게된다. 이런 방법을 **패딩(padding)**이라고 부른다. 특히 입력과 동일한 모양의 특성맵을 만들기 위해 입력 주위에 0을 채워넣는 패딩을 **same padding**이라고 부른다. 그와 달리 입력 배열만으로 특성맵을 만드는 것을 **valid padding**이라고 부른다.

* 패딩을 왜 할까?

패딩을 적용하지 않을 때 (4,4) 입력의 네 개 모서리 값(3, 7, 3, 8)이 합성곱 계산에 포함되는 회수는 단 한 번이다. 하지만 (4,4) 배열의 복판에 있는 네 개 값(4,8,5,1)은 4번의 합성곱 계산에 모두 포함된다. 즉 패딩을 하지 않으면 이미지의 중심부에 있는 값들에 비해 주변에 있는 값들이 특성맵을 만들 때 소홀히 취급된다.



432-3쪽 그림을 가지고 연습해보자.

432쪽 그림은 패딩이 없는 (4,4) 입력의 경우('valid padding')이다. a,b,c가 합성곱에 몇 번씩 들어가는가?

433쪽 그림은 (4,4) 입력에 1개 패딩을 더해 (6,6) 입력을 만든 것이다. (4,4) 특성맵이 만들어진다('same padding'). 이번에는 a,b,c가 합성곱에 몇 번씩 들어가는가?

same padding을 사용한 Conv2D 클래스는 아래와 같이 입력하면 된다.

    keras.layers.Conv2D(10, kernel_size = (3,3), activation = 'relu', padding = 'same')

b) stride

합성곱 연산을 할 때 왼쪽에서 오른쪽으로, 그리고 위에서 아래로 이동하는 칸의 수를 stride라고 한다. 기본값은 1이다.

스트라이드 값을(실제 바꿀 일은 드물겠지만) 변경하고 싶다면 아래와 같이 하면 된다.

        keras.layers.Conv2D(10, kernel_size = (3,3), activation = 'relu', padding = 'same', strides = 2)

3) 풀링(pooling)

**풀링**은 특성맵의 가로세로 크기를 줄임으로써 계산량과 메모리 사용량을 줄이고 매개변수의 수도 줄이는 방법이다. 합성곱 필터와 달리 가중치가 없다. 그리고 패딩을 적용하지 않는다(즉 겹치지 않게 이동한다). 가령 (2,2)**최대풀링(max pooling)**의 경우 특성맵에 있는 (2,2) 네 값 가운데 가장 큰 값 하나를 고른다. 그리고 패딩없이 즉 stride값을 2로 두고 이동한다. 보통 최대풀링을 이용하며 평균풀링도 있다.



435쪽 위의 그림부터 보자. (3,3) 필터 세 개(다른 색깔로 나타냈다)를 (4,4) 입력에 적용하면 (2,2) 특성맵 세 개가 생긴다. 즉 특성맵의 크기는 (2,2,3)이다. 세 특성맵에 (2,2) 풀링을 적용하면 특성맵의 크기는 (1,1,3)이 된다.

이번에는 같은 페이지 아래쪽 그림을 보자. (4,4) 특성맵이 있다. (2,2) 최대 풀링을 적용하면 9,7,8,6 네 숫자로 이루어진 (2,2) 특성맵이 생긴다.

케라스에서는 최대풀링은 'MaxPooling2D()', 평균풀링은 'AveragePooling2D()'를 이용하면 된다. 괄호 속에는 풀링의 크기부터 시작해서 strides값, padding 종류 등을 지정할 수 있다.

    keras.layers.MaxPooling2D(2)

    keras.layers.MaxPooling2D(2, strides=2, padding='valid')



### 8-1-3 CNN의 전체 구조

437쪽 그림과 이어지는 설명을 보고 지금까지 공부한 것을 복습하자.

*** 컬러이미지를 사용한 합성곱**

컬러이미지 입력은 너비, 높이 뿐 아니라 RGB 색깔을 나타내는 깊이(채널이라고도 부른다)까지 3차원이다. 이럴 경우 합성곱을 위한 필터도 3차원이 되어야한다. 가령 입력이 (4,4,3)이면 $3 \times 3$필터의 크기도 (3,3,3)이어야한다. 이럴 경우 27개의 가중치와 절편을 이용해서 합성곱을 계산한다. **따라서 3차원 입력의 경우에도 합성곱을 계산한 결과는 단 하나의 값**이다.

* 흑백이미지 데이터에 Conv2D 층을 이용하려면 `reshape()`을 이용해서 3차원으로 바꾸어야한다.

***합성곱-풀링-합성곱 층**

이번에는 합성곱-풀링 층을 통과한 결과 특성맵의 크기가 (4,4,5)라고 해보자. (퀴즈 : 필터가 몇 개인가?)

(4,4,5) 특성맵에 이어서 또 합성곱 층이 있는 경우를 나타낸 그림이 440쪽 그림이다. 너비, 높이가 3이라면 입력 깊이와 필터 깊이가 같아야하므로 필터의 크기는 (3,3,5)다. 즉 45개의 가중치와 절편으로 값을 하나 만든다. 그런 (3,3,5) 필터가 10개라면 특성맵은 (2,2,10)이 된다.

442쪽 마무리를 잘 읽고 확인문제를 풀어보자.